# Importing Libaries and Data Sets

In [118]:
import pandas as pd

vacc = pd.read_csv("Orignal Data/Vaccination_Rates_by_Neighborhood.csv")
income = pd.read_csv("Orignal Data/LA_Median_Income_By_Neighborhood.csv")
population = pd.read_csv("Orignal Data/Population_Density_by_Neighborhood.csv")
print(len(vacc["CITY"].unique()))
vacc.head()

89


,CITY_TYPE,CITY,COMMUNITY,LABEL,City_Community,Cases,Case_Rate,Deaths,Death_Rate,Percent_of_People_with_1__Dose,Percent_of_People_Fully_Vaccinated
0,City,Agoura Hills,,City of Agoura Hills,City of Agoura Hills,4220,20208.0,22,105.0,0.85,0.80
1,City,Alhambra,,City of Alhambra,City of Alhambra,17044,19653.0,255,294.0,0.87,0.81
2,City,Arcadia,,City of Arcadia,City of Arcadia,8145,14103.0,165,286.0,0.91,0.84
3,City,Artesia,,City of Artesia,City of Artesia,4186,24924.0,88,524.0,0.87,0.80
4,City,Avalon,,City of Avalon,City of Avalon,63,1628.0,0,0.0,0.27,0.23


# Convert Dollar Values to Int

In [119]:
income["MEDIAN INCOME"] = income["MEDIAN INCOME"].str.replace('$', '', regex=False).str.replace(',', '', regex=False)
income["MEDIAN INCOME"] = income["MEDIAN INCOME"].astype(int)

population["POPULATION PER SQMI"] = population["POPULATION PER SQMI"].str.replace(',', '', regex=False)
population["POPULATION PER SQMI"] = population["POPULATION PER SQMI"].astype(int)

#Fix spelling of one Neighborhood
# "La Cañada Flintridge"
income["NEIGHBORHOOD"] = income["NEIGHBORHOOD"].replace("La Cañada Flintridge", "La Canada Flintridge")
population["NEIGHBORHOOD"] = population["NEIGHBORHOOD"].replace("La Cañada Flintridge", "La Canada Flintridge")

#add commulities to cities
vacc.loc[vacc['CITY'] == 'Los Angeles', 'CITY'] = vacc.loc[vacc['CITY'] == 'Los Angeles', 'COMMUNITY']


income.head()

,rank,NEIGHBORHOOD,MEDIAN INCOME
0,1,Bel-Air,207938
1,2,Hidden Hills,203199
2,3,Rolling Hills,184777
3,4,Beverly Crest,169282
4,5,Pacific Palisades,168008


# Find What "Neighborhoods" Are In Both Datasets

In [120]:
#Find "Neighborhoods" found in both datasets
common_values = set(income["NEIGHBORHOOD"].unique().tolist()).intersection(set(vacc["CITY"].unique().tolist()))

#Count the number of common values
count = len(common_values)
print(count)
print(common_values)

173
{'Cudahy', 'Porter Ranch', 'Downtown', 'Valley Village', 'Agoura Hills', 'Downey', 'Commerce', 'La Habra Heights', 'Bradbury', 'Claremont', 'La Mirada', 'Encino', 'Calabasas', 'Elysian Park', 'Panorama City', 'South Pasadena', 'Rosemead', 'Walnut', 'Signal Hill', 'Santa Fe Springs', 'La Canada Flintridge', 'Arleta', 'Bell', 'Diamond Bar', 'Huntington Park', 'Arcadia', 'Northridge', 'Westlake', 'Monrovia', 'Baldwin Park', 'El Sereno', 'Boyle Heights', 'Valley Glen', 'Granada Hills', 'Lincoln Heights', 'Wilmington', 'Azusa', 'Vermont Vista', 'Pomona', 'Artesia', 'La Verne', 'Jefferson Park', 'Hermosa Beach', 'Pacific Palisades', 'Venice', 'West Adams', 'Rancho Palos Verdes', 'Harbor City', 'Sylmar', 'Hawthorne', 'Gardena', 'Eagle Rock', 'Rolling Hills Estates', 'Lomita', 'Lynwood', 'San Marino', 'Beverly Crest', 'Santa Clarita', 'San Pedro', 'Lawndale', 'Pacoima', 'Leimert Park', 'Vermont Square', 'Harbor Gateway', 'El Monte', 'Studio City', 'Vermont Knolls', 'Hidden Hills', 'Canoga 

# Determine What "Neighborhoods" Are In The Vaccination Dataset, But Not In the Income Dataset

In [121]:
# Convert coumns to sets
set1 = set(vacc["CITY"].unique().tolist())

# Find values not in both sets
values_not_in_both = list(set1.symmetric_difference(common_values))

# Print the list of values not found in both lists
print(values_not_in_both)


['Gramercy Place', 'Harbor Pines', 'South Carthay', 'Vernon Central', 'Mandeville Canyon', 'View Heights', 'Park La Brea', 'Thai Town', 'Country Club Park', 'Mt. Washington', 'Cadillac-Corning', 'Angeles National Forest', 'Wilshire Center', 'Century Palms/Cove', 'Wellington Square', 'Silverlake', 'St Elmo Village', 'Palisades Highlands', 'Temple-Beaudry', 'Little Bangladesh', 'Exposition', 'Toluca Terrace', 'Angelino Heights', 'Playa Del Rey', 'Victoria Park', 'University Hills', 'Reseda Ranch', 'Melrose', 'Alsace', 'Lakeview Terrace', 'Reynier Village', 'Regent Square', 'Historic Filipinotown', 'Unincorporated', 'Little Armenia', 'Marina Peninsula', 'Crestview', 'Lafayette Square', 'Crenshaw District', 'Sycamore Square', 'Toluca Woods', 'Baldwin Hills', 'Mid-city', 'Miracle Mile', 'Longwood', 'Brookside', 'Cloverdale/Cochran', 'Little Tokyo', 'West Vernon', 'Figueroa Park Square', 'Central', 'Wholesale District', 'Faircrest Heights', 'Bel Air']


# Trim Both Datasets So They Only have "Neighborhoods" That Are In Common

In [122]:
#Filter Vaccination Dataset
vacc = vacc[vacc['CITY'].isin(common_values)]

#merge income and population
income = pd.merge(income, population, on="NEIGHBORHOOD")
#     drop columns
income = income[["NEIGHBORHOOD", "MEDIAN INCOME", "POPULATION PER SQMI"]]

#Filter Income and Population Dataset
income = income[income['NEIGHBORHOOD'].isin(common_values)]

#Print the sizes of the filtered DataFrame
print(len(vacc))
print(len(income))
income.head()

173
173


,NEIGHBORHOOD,MEDIAN INCOME,POPULATION PER SQMI
1,Hidden Hills,203199,1100
2,Rolling Hills,184777,893
3,Beverly Crest,169282,1318
4,Pacific Palisades,168008,1048
5,Palos Verdes Estates,167344,2822


# Add "Income Level" Column (High, Medium, Low)

In [123]:
#add income levels
# high is 75675 and above
# medium is 52146 and above

#bins and labels of the ranges
bins = [0, 52146, 75675, float('inf')]
labels = ['low', 'medium', 'high']

#Apply bins to df2 and make a column for "income level"
income['income level'] = pd.cut(income['MEDIAN INCOME'], bins=bins, labels=labels, right=False)
income.head()

,NEIGHBORHOOD,MEDIAN INCOME,POPULATION PER SQMI,income level
1,Hidden Hills,203199,1100,high
2,Rolling Hills,184777,893,high
3,Beverly Crest,169282,1318,high
4,Pacific Palisades,168008,1048,high
5,Palos Verdes Estates,167344,2822,high


# Save CSVs

In [124]:
# Save the modified DataFrame as a CSV file
vacc.to_csv('Cleaned Data/Vaccination_Rates.csv', index=False)
income.to_csv('Cleaned Data/Median_Income.csv', index=False)